# Decision Tree Implementation on the HR_Churn Dataset

>This project uses a dataset from Kaggle and attemps to predict whether or not a certain employee will leave the company
or not. 
The data set analysis is performed in HR_Churn Project and will not be provided here.
The Project uses Decision Trees as an algorithm to predict whether or not a certain employee will leave the company.
The project starts by using sklearn to make the prediction. 
It then provides an imlementation of Decision Trees from first principles i.e without Decision Trees to attemp to re-create the same prediction.

## Conventions 

**X**  - Imput Matrix of Size NxD where N is the number of samples and D is the number of features

**Y** - Represents the targets. For binary classification, the outputs will be either 0 or 1.

In [1]:
import numpy as np
import pandas as pd 
from datetime import datetime 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,confusion_matrix

In [2]:
df = pd.read_csv("C:/Users/Botev/Desktop/logreg/HR_comma_sep.csv")

### Data preparation 

> Normalising the Data

In [3]:
df["sat_level_norm"] = (df["satisfaction_level"] - df["satisfaction_level"].mean())/(df["satisfaction_level"].max()-df["satisfaction_level"].min())

In [4]:
df["last_evaluation_norm"] = (df["last_evaluation"] - df["last_evaluation"].mean())/(df["last_evaluation"].max()-df["last_evaluation"].min())

In [5]:
df["average_montly_hours_norm"] = (df["average_montly_hours"] - df["average_montly_hours"].mean())/(df["average_montly_hours"].max()-df["average_montly_hours"].min())

In [6]:
df["number_project_norm"] = (df["number_project"] - df["number_project"].mean())/(df["number_project"].max()-df["number_project"].min())

In [7]:
# Encoding the salary Column
def salary_encode(salary):
    if salary =="low":
        return 1
    elif salary=="medium":
        return 2
    else:
        return 3

In [8]:
df["salary_encoded"] = df["salary"].apply(salary_encode)

In [9]:
X = df[["satisfaction_level","last_evaluation","number_project","average_montly_hours","time_spend_company","Work_accident","promotion_last_5years"]]
y = df["left"]

In [17]:
# Spliting the data into training and testing. The training and testing data sizes are equal.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=101)

In [18]:
dtree = DecisionTreeClassifier()

In [24]:
fit = dtree.fit(X_train,y_train)

In [20]:
predictions = dtree.predict(X_test)

In [22]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.99      0.98      0.98      5708
          1       0.93      0.96      0.95      1792

avg / total       0.97      0.97      0.97      7500



In [21]:
print(confusion_matrix(y_test,predictions))

[[5576  132]
 [  63 1729]]


In [25]:
print "Accuracy:%d"%(int(fit.score(X_test,y_test)*100))+'%'

Accuracy:97%


In [26]:
print "Accuracy:%d"%(int(fit.score(X_train,y_train)*100))+'%'

Accuracy:99%


> The Sklearn Decision Tree implementation yields a training accuracy of 99% and testing accuracy of 97%.

## HR_Churn Project - Decision Tree Implementation Without Using Sklearn

> This part contains an implementation for Decision Trees Algorithm without Sklearn

## Data-Maipulating functions for HR_Churn Dataset

### Helper Functions 

In [ ]:
def encode(X,Y):
    """
    Performs one hot encoding for the salary column of dataset
    """
    X_mat = X.as_matrix()#conver to matrixes 
    Y_mat = Y.as_matrix()
    N,D = X_mat.shape
    X2 = np.zeros((N,D+3))# introducing a second matrix. 
    #This matrix has 3 more column than the original so that the encoding 
    #can be performed
    X2[:,0:D-1] = X_mat[:,0:D-1]
    
    
    for n in xrange(N):
        i=int(X_mat[n,D-1])
        X2[n,D-1+i]=1
    
    return X2,Y_mat

In [ ]:
def salary_encode(salary):
    """
    Converts the categorical variables for the salary into numerical
    """
    if salary =="low":
        return 1
    elif salary=="medium":
        return 2
    else:
        return 3

In [18]:
def get_data_hr_churn():
    """
    Reads the data first, then normalises certain columns of the dataset
    Applies salary_encode to get rid of the categorical variables,then applies
    encode to perform one hot encoding. Returns the X and Y as numpy arrays in
    2D form, similar to a matrix.
    """
    df = pd.read_csv("C:/Users/Botev/Desktop/logreg/HR_comma_sep.csv")
    #first column normalisation
    df["sat_level_norm"] = (df["satisfaction_level"] - 
    df["satisfaction_level"].mean())/(df["satisfaction_level"].max()-
                                              df["satisfaction_level"].min())
    #second column normalisation
    df["average_montly_hours_norm"] = (df["average_montly_hours"] - 
    df["average_montly_hours"].mean())/(df["average_montly_hours"].max()-
                                              df["average_montly_hours"].min())
    #third column normalisation
    df["time_spend_company_norm"] = (df["time_spend_company"] - 
    df["time_spend_company"].mean())/(df["time_spend_company"].max()-
                                      df["time_spend_company"].min())
    #forth column normalisation
    
    df["number_project_norm"] = (df["number_project"] - 
    df["number_project"].mean())/(df["number_project"].max()-
                                  df["number_project"].min())
    
    
    df["salary_encoded"] = df["salary"].apply(salary_encode)
    #encodes the salary creating 
    
    
    X_new= df[["satisfaction_level","last_evaluation","Work_accident",
    "promotion_last_5years","number_project_norm","time_spend_company_norm",
    "average_montly_hours_norm","salary_encoded"]]
    Y_new = df["left"]
    
    Xout,Yout = encode(X_new,Y_new)
    
    return Xout,Yout

In [ ]:
def entropy(y):
    """
    Calculates entropy
    """
    # assume y is binary - 0 or 1
    N = len(y)
    s1 = (y == 1).sum()
    if 0 == s1 or N == s1:
        return 0
    p1 = float(s1) / N
    p0 = 1 - p1
    return -p0*np.log2(p0) - p1*np.log2(p1)

### Implementation 

In [15]:
class DecisionTree:
    def __init__(self, depth=0, max_depth=None):
        # print 'depth:', depth
        self.depth = depth
        self.max_depth = max_depth

    def fit(self, X, Y):
        """
        Checks for bases cases and information gain eqaul to 0.
        Finds the attribute where the split occurs(self.col) as well as 
        the value at which the spliting occurs(self.split). 
        Checks if the maximum depth has been reached. If yes, aggregate all 
        values to the left and to the right. If not continue splitting. 
        Every time a split has occured the depth needs to be incremented by
        one. 
        
        
        """
        
        """
        In this case we are checking for two main cases:
        Case1 -  When we have a single sample
        Case2 -  When we have a single class
        In both of the cases no  predictions will be made
        
        """
        if len(Y) == 1 or len(set(Y)) == 1:
        
            #chacks for two base cases: only 1 sameple and only one class
            #We do not make predictions here 
            self.col = None
            self.split = None
            self.left = None
            self.right = None
            self.prediction = Y[0]

        else:
            D = X.shape[1]
            cols = range(D)

            max_info_gain = 0
            best_col = None
            best_split = None
            for col in cols:
                info_gain, split = self.find_split(X, Y, col)
                if info_gain > max_info_gain:
                    max_info_gain = info_gain
                    best_col = col
                    best_split = split

            if max_info_gain == 0:
                # no further splits will be performed
                self.col = None
                self.split = None
                self.left = None
                self.right = None
                self.prediction = np.round(Y.mean())
            else:
                self.col = best_col#this is the attribute we split on
                self.split = best_split#this is the value from that attribute
                #we have split on

                if self.depth == self.max_depth:
                    self.left = None
                    self.right = None
                    self.prediction = [
                        np.round(Y[X[:,best_col] < self.split].mean()),
                        np.round(Y[X[:,best_col] >= self.split].mean()),
                    ]
                    """
                    self.prediction contains the actual predictions 
                    can be called by self.prediction[0] for left and 
                    self.predictin[1] for right slips
                    At the moment it has a 50% confidence interval i.e
                    if there are more 1ns than 0s it will classify it as
                    1 and vice versa. An additinal method can be implemented
                    to increase the confidence interval to say 75%. This 
                    additional method will output one if its input is say
                    over 0.75 and 0 if less.
                    """
                else:
                    # print "best split:", best_split
                    left_idx = (X[:,best_col] < best_split)
                    #print(left_idx)
                    #left_idx - variable that holds true or false for each 
                    #sample in the best_col. 
                    Xleft = X[left_idx]
                    #print(Xleft)
                    Yleft = Y[left_idx]
                    
                    #Xleft and Yleft contain the data that was split
                    #to the left of the condition. self.left then becomes
                    #child node which is still of class TreeNode with 
                    #the depth incremented by 1
                    
                    self.left = DecisionTree(self.depth + 1, self.max_depth)
                    self.left.fit(Xleft, Yleft)

                    right_idx = (X[:,best_col] >= best_split)
                    Xright = X[right_idx]
                    Yright = Y[right_idx]
                    self.right = DecisionTree(self.depth + 1, self.max_depth)
                    self.right.fit(Xright, Yright)

    def find_split(self, X, Y, col):
        """
        Returns the information gain and the value at which the best split has
        occured for that particular column. 
        The split is the middle point b/n 2 consequitive values. It is a 
        number.
        Starts by arranging all samples in a column in an ascending order.
        It then checks for the indexes in y where y changes from 0 to 1.
        
        """
        
        x_values = X[:, col]
        sort_idx = np.argsort(x_values)
        x_values = x_values[sort_idx]
        y_values = Y[sort_idx]

        boundaries = np.nonzero(y_values[:-1] != y_values[1:])[0]
        best_split = None
        max_info_gain = 0
        for b in boundaries:
            split = (x_values[b] + x_values[b+1]) / 2
            info_gain = self.information_gain(x_values, y_values, split)
            if info_gain > max_info_gain:
                max_info_gain = info_gain
                best_split = split
        return max_info_gain, best_split

    def information_gain(self, x, y, split):
        """
        Computes the information gain on a given split value 
        """
        
        y0 = y[x < split]
        y1 = y[x >= split]
        N = len(y)
        y0len = len(y0)
        if y0len == 0 or y0len == N:
            return 0
        p0 = float(len(y0)) / N
        p1 = 1 - p0 
        return entropy(y) - p0*entropy(y0) - p1*entropy(y1)

    def single_sample_pred(self, x):
        """
        Outputs the prediction on a single sample for a particular 
        attribute(column)
        """
        # use "is not None" because 0 means False
        if self.col is not None and self.split is not None:
            feature = x[self.col]
            if feature < self.split:
                if self.left:
                    p = self.left.single_sample_pred(x)
                else:
                    p = self.prediction[0]
            else:
                if self.right:
                    p = self.right.single_sample_pred(x)
                else:
                    p = self.prediction[1]
        else:
            p = self.prediction
        return p

    def predict_all_samples(self, X):
        """
        Calls predict_one on each sample. 
        
        """
        N = len(X)
        P = np.zeros(N)
        for i in xrange(N):
            P[i] = self.single_sample_pred(X[i])
        return P
    
    def score(self,X,Y):
        """
        Compares all the data points in the predicted class P to the 
        input class Y, and finds the mean. This then forms the accuracy of the 
        model.
        """
        P = self.predict_all_samples(X)
        return np.mean(P==Y)

In [19]:
if __name__ == '__main__':
    X, Y = get_data_hr_churn()#uses the data from the churn project

    

    # split the data
    Ntrain = len(Y) / 2
    Xtrain, Ytrain = X[:Ntrain], Y[:Ntrain]
    Xtest, Ytest = X[Ntrain:], Y[Ntrain:]
    
    model = DecisionTree(0,10)#Initialise a Decision Tree with a maximum depth of 10 
    t0 = datetime.now()
    model.fit(Xtrain, Ytrain)

    #Training Accuracy and time to compute training accuracy 
    print "Train accuracy:", model.score(Xtrain, Ytrain)

    t0 = datetime.now()
    #print("Prediction: ",model.predict(Xtest))
    #Testing Accuracy and time to compute testing accuracy 
    print "Test accuracy:", model.score(Xtest, Ytest)

Train accuracy: 0.98546472863
Test accuracy: 0.972933333333


## Conslusion 

> The Decision Tree implementation without Sklearn yeilds training accuracy of 98.5% and testing accuracy of 97.3%. The values 
are overall very similar to those generated by the implementation using Sklearn provided above. 
In general increasing the depth of the decision tree results in higher accuracy as expected.